In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from openpyxl import load_workbook
import csv
import pandas as pd

# Step 1. Load the data

* **`RPPA:`** Reverse-Phase Protein Array, representing the gene-(signaling) protein expression data
* **`GCP:`** Global Chromatin Profiling, quantifying the post-translational modifications on histone proteins 

In [ ]:
rppa_file = os.path.join('dataset/rppa/','****.xlsx')
gcppbs_file = os.path.join('dataset/gcp/','****.xlsx')

# rppa_sheet = pd.read_csv(rppa_file).drop('antibody', axis=1)
# print(rppa_sheets.keys())
rppa_sheets = pd.read_excel(rppa_file, sheet_name=None)
rppa_sheet = rppa_sheets['MDD_RPPA_Level3_annotated']
rppa_sheet

In [ ]:
rawmergeDf = rppa_sheet.set_index('Protein')
tmpS=pd.Series(rawmergeDf.columns)
rawmergeDf.columns=tmpS.str.split('_').apply(lambda L:"_".join(L[:2])).values
rawmergeDf.columns

In [5]:
from scipy import stats
#time_condition='^48hr'
time_conditions=['_4','_8','_24','_48']
myL=[]
for time_condition in time_conditions:
    time_m=rawmergeDf.columns.str.endswith(time_condition)
#     print(time_m)
    #rawmergeDf
    conditions=rawmergeDf.columns[time_m&(~rawmergeDf.columns.str.contains('Ctrl'))].unique()
    print (conditions)
    myDict={}
    for condition in conditions:
        A=rawmergeDf.loc[:,rawmergeDf.columns.str.endswith(condition)]
#         B=rawmergeDf.loc[:,rawmergeDf.columns.str.contains('ctrl_0')]
        B=rawmergeDf.loc[:,rawmergeDf.columns.str.endswith('Ctrl'+time_condition)]
        #rawmergeDf.loc['CASP7']
        t,p=stats.ttest_ind(A,B,axis=1)
        #geneP_S=pd.Series(index=rawmergeDf.index,data=p)
        fc=A.mean(axis=1)-B.mean(axis=1)
        """multi=pd.MultiIndex.from_arrays([mergedPvalDf.index.to_frame()[0].str.split('_').str[1],
                           mergedPvalDf.index.get_level_values(1)
                          ])"""
        tmpDf=pd.DataFrame({'fc':fc,'p':p,'t':t},index=rawmergeDf.index)
        myDict[condition]=tmpDf#[tmpDf['p']<0.05]
    mergedPvalDf=pd.concat(myDict.values(),
                           keys=pd.Series(list(myDict.keys())).str.split('_').str[0].values)
    #mergedPvalDf.index=mergedPvalDf.index.str.split('_').str[0]
    myL.append(mergedPvalDf)

Index(['BMP2_4', 'IFNG_4', 'TGFB_4', 'HGF_4', 'OSM_4', 'EGF_4'], dtype='object')
Index(['BMP2_8', 'IFNG_8', 'TGFB_8', 'HGF_8', 'OSM_8', 'EGF_8'], dtype='object')
Index(['BMP2_24', 'IFNG_24', 'TGFB_24', 'HGF_24', 'OSM_24', 'EGF_24'], dtype='object')
Index(['BMP2_48', 'IFNG_48', 'TGFB_48', 'HGF_48', 'OSM_48', 'EGF_48'], dtype='object')


In [8]:
time_conditions_h = ['hour' + h for h in time_conditions]
mergedPvalDf=pd.concat(myL,axis=1,keys=time_conditions_h)
mergedPvalDf.to_excel("pval_all_rppa.xlsx")

In [9]:
mergedPvalDf

hour_4                         hour_8                       \
                      fc         p          t        fc         p          t   
     Protein                                                                   
BMP2 ABL1      -0.009419  0.746565  -0.346301  0.010297  0.530471   0.685890   
     ACACApS79  0.077289  0.375558   0.996147  0.055869  0.639585   0.505820   
     ACACA      0.633688  0.008281   4.859661  0.680632  0.000595   9.855555   
     ACTB       0.078077  0.284381   1.235100  0.185688  0.011837   4.384057   
     ADAR       0.000058  0.999659   0.000455  0.031192  0.749298   0.342381   
...                  ...       ...        ...       ...       ...        ...   
EGF  YAP1      -0.204034  0.330977  -1.105455 -0.320344  0.016384  -3.981254   
     YBX1pS102  0.780322  0.000005  32.694463  0.750876  0.000086  16.156085   
     YWHAB     -0.095544  0.110982  -2.039699 -0.093693  0.013728  -4.197094   
     YWHAZ     -0.008907  0.841044  -0.213957 -0.053431  0.117820  -1.987252   
     ZAP70     -0.132021  0.113417  -2.020627 -0.125867  0.048659  -2.803073   

                 hour_24                        hour_48                       
                      fc         p          t        fc         p          t  
     Protein                                                                  
BMP2 ABL1       0.023789  0.574663   0.610255  0.071314  0.343437   1.073642  
     ACACApS79 -0.021280  0.846576  -0.206377  0.128604  0.317783   1.140353  
     ACACA      0.687672  0.012825   4.282197  0.596771  0.110555   2.043091  
     ACTB       0.432777  0.003723   6.069143  0.670866  0.013976   4.174836  
     ADAR       0.138112  0.498809   0.742890  0.739005  0.038145   3.046872  
...                  ...       ...        ...       ...       ...        ...  
EGF  YAP1      -0.894787  0.012408  -4.324004 -2.033225  0.001549  -7.676414  
     YBX1pS102  1.167023  0.000037  19.931804  1.002398  0.000156  13.877654  
     YWHAB     -0.162509  0.185602  -1.596548 -0.367276  0.009211  -4.714153  
     YWHAZ     -0.197220  0.028975  -3.334832 -0.293756  0.008894  -4.761776  
     ZAP70     -0.014297  0.882965  -0.156846 -0.117863  0.178834  -1.628103  

[1770 rows x 12 columns]

In [10]:
ligand_min = mergedPvalDf.groupby(level=['Protein'],axis=0).min()

In [11]:
ligand_min

hour_4                        hour_8                      \
                 fc         p         t        fc         p         t   
Protein                                                                 
ABL1      -0.071919  0.087555 -2.250932 -0.079951  0.078500 -2.350177   
ACACA      0.300844  0.002434  2.238141  0.346848  0.000457  3.576798   
ACACApS79 -0.482309  0.056315 -0.778100  0.024917  0.158201  0.232581   
ACTB       0.012234  0.077878  0.157801  0.073282  0.003734  1.479511   
ADAR      -0.062241  0.101766 -2.116291 -0.014355  0.495983 -0.108708   
...             ...       ...       ...       ...       ...       ...   
YAP1pS127 -0.253568  0.039995 -2.998658 -0.544808  0.007311 -5.034353   
YBX1pS102  0.317199  0.000005  9.272837  0.331333  0.000060  7.016739   
YWHAB     -0.140724  0.027519 -3.390446 -0.146803  0.013364 -4.230609   
YWHAZ     -0.044897  0.462236 -0.812220 -0.053431  0.117820 -1.987252   
ZAP70     -0.133990  0.018005 -3.869241 -0.178730  0.007221 -5.052026   

            hour_24                       hour_48                      
                 fc         p         t        fc         p         t  
Protein                                                                
ABL1      -0.047361  0.103227 -1.963663 -0.042217  0.343437 -0.612114  
ACACA      0.111401  0.012481  0.986883  0.133842  0.013987  0.515097  
ACACApS79 -0.236249  0.037950 -2.589868  0.122431  0.019812  1.140353  
ACTB       0.181127  0.002233  2.323627  0.185483  0.001950  1.000403  
ADAR       0.105648  0.346477  0.716470  0.651324  0.015546  2.526139  
...             ...       ...       ...       ...       ...       ...  
YAP1pS127 -0.827437  0.001271 -8.086033  0.121193  0.001018  1.786746  
YBX1pS102  0.597449  0.000037  8.479956  0.330785  0.000156  2.414792  
YWHAB     -0.238006  0.069866 -2.457675 -0.367276  0.009211 -4.714153  
YWHAZ     -0.199747  0.028975 -3.334832 -0.293756  0.008894 -4.761776  
ZAP70     -0.065444  0.572473 -0.613907 -0.145859  0.016849 -3.947787  

[295 rows x 12 columns]

In [12]:
hour_min = ligand_min.groupby(level=1, axis=1).min()

In [13]:
protein_significants = hour_min[hour_min['p']<0.05].sort_values(by=['p'], ascending=False).drop(['fc', 't'], axis=1)
protein_significants

,p
Protein,
RPA2pS4S8,4.886948e-02
CHEK2,4.607508e-02
ERCC1,4.395053e-02
RPS6KA1,4.314620e-02
BID,4.235510e-02
...,...
HIF1A,9.166431e-07
DUSP4,7.442644e-07
CCNB1,4.328352e-07


In [15]:
protein_significants.to_excel("pval_005_rppa.xlsx")

In [52]:
gcp_sheet = pd.read_excel(gcppbs_file, sheet_name='two_NaN_histone_removed_T')
gcp_sheet

,histone,H3K4me0,H3K4me1,H3K4me2,H3K4me3,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,...,H4(4to17)K12ac1K16ac1me0,H4(4to17)K8ac1K12ac1K16ac1me0,H4(4to17)K5ac1K8ac1K12ac1me0,H4(4to17)K5ac1K8ac1K16ac1me0,H4(4to17)K5ac1K8ac1K12ac1K16ac1me0,H4(20to23)K20me0,H4(20to23)K20me1,H4(20to23)K20me2,H4(20to23)K20me3,H4(68-78)AltNorm
0,BMP2_24,-0.969402,-1.380645,-1.827062,-2.426360,-5.255588,-0.694744,-1.051102,-0.755358,-2.208249,...,-3.836672,-2.376208,-1.749209,-2.258351,-1.816227,-0.682292,-0.972047,-1.485506,-1.155143,0.210474
1,BMP2_24,-0.899803,-1.310074,-1.748623,-2.397195,-5.659498,-0.733472,-1.119252,-0.895467,-2.538554,...,-2.969416,-1.705970,-1.146616,-1.767997,-1.295829,-0.388577,-0.704941,-0.858888,-1.522616,0.156758
2,BMP2_24,-0.254084,-0.615388,-1.228978,-2.008718,-5.495082,-0.673366,-0.357395,-0.184616,-2.054138,...,-2.732027,-1.908453,-0.944077,-2.013174,-1.339216,-0.859446,-0.750306,-0.547187,-0.614954,0.635867
3,BMP2_4,-0.797550,-1.278252,-1.416830,-2.219870,-5.396007,-1.171865,-0.952779,-0.878783,-1.941579,...,-2.603124,-1.003388,-0.851667,-1.603124,-1.322916,-2.350695,-1.056358,-0.732086,-1.683270,-0.273677
4,BMP2_4,-0.439043,-1.050139,-1.530612,-2.241806,-5.281459,-1.007038,-0.912903,-0.261658,-2.058129,...,-3.350256,-1.248569,-0.913087,-1.838916,-1.382136,-2.994881,-0.992479,-0.767859,-1.415540,0.142895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,TGFb_48,-0.632747,-1.207177,-1.221169,-2.011055,-5.259965,0.003666,-0.343092,-0.262720,-1.380322,...,-3.356485,-1.853985,-1.377125,-2.271596,-0.961224,-0.414326,-0.472361,-1.595135,-1.799109,0.309497
72,TGFb_48,-0.992086,-1.405334,-1.818286,-2.301809,-5.259965,-0.096163,-0.398864,-0.515418,-1.638274,...,-2.608073,-1.465585,-0.569894,-1.730430,-0.961224,-0.112342,-0.353591,-1.146374,-1.563830,-0.054227
73,TGFb_8,-0.951989,-1.260621,-1.283264,-2.316241,-5.400847,-1.024402,-1.092956,-0.798695,-2.045269,...,-2.697035,-1.271986,-0.696701,-1.602420,-1.077658,-2.047884,-0.582486,-0.645393,-1.108060,-0.161121
74,TGFb_8,-1.087854,-1.512375,-1.530698,-2.425977,-5.906808,-1.085143,-1.265821,-0.824732,-1.775268,...,-2.861606,-1.671866,-1.242984,-1.967569,-1.777425,-2.250059,-0.921661,-0.749944,-0.682765,0.156851


In [53]:
gcp_sheet_T = gcp_sheet.set_index('histone').T
gcp_sheet_T.index.name = gcp_sheet_T.columns.name
gcp_sheet_T.columns.name = None
gcp_sheet_T

,BMP2_24,BMP2_24,BMP2_24,BMP2_4,BMP2_4,BMP2_4,BMP2_48,BMP2_48,BMP2_8,BMP2_8,...,TGFb_24,TGFb_24,TGFb_4,TGFb_4,TGFb_4,TGFb_48,TGFb_48,TGFb_8,TGFb_8,TGFb_8
histone,,,,,,,,,,,,,,,,,,,,,
H3K4me0,-0.969402,-0.899803,-0.254084,-0.797550,-0.439043,-0.949751,-0.616780,-1.131222,-0.973947,-0.652746,...,-1.017304,-1.504624,-0.792174,-0.819048,-0.664739,-0.632747,-0.992086,-0.951989,-1.087854,-0.943708
H3K4me1,-1.380645,-1.310074,-0.615388,-1.278252,-1.050139,-0.970071,-1.352532,-1.638505,-1.197797,-0.926891,...,-1.587767,-2.023841,-1.127231,-1.304880,-0.740275,-1.207177,-1.405334,-1.260621,-1.512375,-1.255098
H3K4me2,-1.827062,-1.748623,-1.228978,-1.416830,-1.530612,-1.296827,-1.602961,-1.925617,-1.322990,-1.513021,...,-1.661203,-1.932912,-1.618821,-1.460013,-1.382290,-1.221169,-1.818286,-1.283264,-1.530698,-1.696266
H3K4me3,-2.426360,-2.397195,-2.008718,-2.219870,-2.241806,-2.412855,-2.340077,-2.631456,-2.320042,-2.118178,...,-2.489530,-2.777117,-2.293152,-2.429518,-2.011191,-2.011055,-2.301809,-2.316241,-2.425977,-2.513528
H3K4ac1,-5.255588,-5.659498,-5.495082,-5.396007,-5.281459,-5.645749,-5.546153,-5.579835,-5.570075,-5.282585,...,-6.799838,-6.512329,-5.335094,-5.514504,-5.501349,-5.259965,-5.259965,-5.400847,-5.906808,-5.484959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H4(20to23)K20me0,-0.682292,-0.388577,-0.859446,-2.350695,-2.994881,-2.083068,-0.266455,0.009549,-2.419590,-3.074881,...,-1.423392,-0.934593,-2.821755,-2.559735,-2.228102,-0.414326,-0.112342,-2.047884,-2.250059,-2.424327
H4(20to23)K20me1,-0.972047,-0.704941,-0.750306,-1.056358,-0.992479,-1.162697,-0.013733,-0.156436,-1.366214,-1.264584,...,-1.187325,-0.795145,-1.827881,-1.146505,-0.747778,-0.472361,-0.353591,-0.582486,-0.921661,-0.755970
H4(20to23)K20me2,-1.485506,-0.858888,-0.547187,-0.732086,-0.767859,-0.601170,-0.647106,-0.717432,-1.283715,-0.752952,...,-0.769815,-0.691272,-1.168012,-1.330057,-0.727614,-1.595135,-1.146374,-0.645393,-0.749944,-0.616695


In [62]:
from scipy import stats
#time_condition='^48hr'
rawmergeDf_gcp = gcp_sheet_T
time_conditions=['_4','_8','_24','_48']
myL_gcp=[]
for time_condition in time_conditions:
    time_m=rawmergeDf_gcp.columns.str.endswith(time_condition)
    #rawmergeDf
    conditions=rawmergeDf_gcp.columns[time_m&(~rawmergeDf_gcp.columns.str.contains('ctrl'))].unique()
    print (conditions)
    myDict_gcp={}
    for condition in conditions:
#         print(condition)
        A=rawmergeDf_gcp.loc[:,rawmergeDf_gcp.columns.str.endswith(condition)]
        B=rawmergeDf_gcp.loc[:,rawmergeDf_gcp.columns.str.contains('ctrl_0')]
#         B=rawmergeDf.loc[:,rawmergeDf.columns.str.contains(time_condition+'_ctrl')]
        #rawmergeDf.loc['CASP7']
        t,p=stats.ttest_ind(A,B,axis=1)
        #geneP_S=pd.Series(index=rawmergeDf.index,data=p)
        fc=A.mean(axis=1)-B.mean(axis=1)
        """multi=pd.MultiIndex.from_arrays([mergedPvalDf.index.to_frame()[0].str.split('_').str[1],
                           mergedPvalDf.index.get_level_values(1)
                          ])"""
        tmpDf_gcp=pd.DataFrame({'fc':fc,'p':p,'t':t},index=rawmergeDf_gcp.index)
        myDict_gcp[condition]=tmpDf_gcp#[tmpDf_gcp['p']<0.05]
    mergedPvalDf_gcp=pd.concat(myDict_gcp.values(),
                           keys=pd.Series(list(myDict_gcp.keys())).str.split('_').str[0].values)
    #mergedPvalDf.index=mergedPvalDf.index.str.split('_').str[0]
    myL_gcp.append(mergedPvalDf_gcp)

Index(['BMP2_4', 'EGF_4', 'HGF_4', 'IFNg_4', 'OSM_4', 'TGFb_4'], dtype='object')
Index(['BMP2_8', 'EGF_8', 'HGF_8', 'IFNg_8', 'OSM_8', 'TGFb_8'], dtype='object')
Index(['BMP2_24', 'EGF_24', 'HGF_24', 'IFNg_24', 'OSM_24', 'TGFb_24'], dtype='object')
Index(['BMP2_48', 'EGF_48', 'HGF_48', 'IFNg_48', 'OSM_48', 'TGFb_48'], dtype='object')


In [64]:
myL_gcp[3]

fc         p         t
     histone                                       
BMP2 H3K4me0          -0.150201  0.362895 -1.000734
     H3K4me1          -0.259853  0.132213 -1.797320
     H3K4me2          -0.416534  0.058478 -2.442434
     H3K4me3          -0.315625  0.012928 -3.777087
     H3K4ac1          -0.404569  0.029279 -3.024100
...                         ...       ...       ...
TGFb H4(20to23)K20me0  1.897346  0.008449  4.204937
     H4(20to23)K20me1  0.315347  0.027236  3.087512
     H4(20to23)K20me2 -0.513190  0.033130 -2.916962
     H4(20to23)K20me3 -0.151613  0.560541 -0.623090
     H4(68-78)AltNorm  0.107820  0.415230  0.887961

[378 rows x 3 columns]

In [65]:
mergedPvalDf_gcp=pd.concat(myL_gcp,axis=1,keys=time_conditions)
mergedPvalDf_gcp.to_excel("pval_all_gcp.xlsx")

In [66]:
mergedPvalDf_gcp

_4                            _8            \
                             fc         p         t        fc         p   
     histone                                                              
BMP2 H3K4me0          -0.004981  0.968629 -0.040996 -0.119874  0.218694   
     H3K4me1           0.136179  0.296401  1.143483  0.166434  0.193528   
     H3K4me2          -0.067001  0.618214 -0.525297 -0.145759  0.324901   
     H3K4me3          -0.121369  0.056284 -2.360005 -0.119914  0.145946   
     H3K4ac1          -0.282647  0.076391 -2.137712 -0.262415  0.075896   
...                         ...       ...       ...       ...       ...   
TGFb H4(20to23)K20me0 -0.375851  0.355423 -1.001109 -0.080077  0.832570   
     H4(20to23)K20me1 -0.512397  0.079486 -2.108972 -0.025049  0.820066   
     H4(20to23)K20me2 -0.217663  0.240906 -1.301245  0.186887  0.133062   
     H4(20to23)K20me3  0.167440  0.484958  0.744043  0.598257  0.028940   
     H4(68-78)AltNorm -0.038067  0.661716 -0.459997  0.082620  0.508324   

                                      _24                           _48  \
                              t        fc         p         t        fc   
     histone                                                              
BMP2 H3K4me0          -1.373547  0.016037  0.929477  0.092283 -0.150201   
     H3K4me1           1.464009  0.133631  0.535865  0.656481 -0.259853   
     H3K4me2          -1.072077 -0.253799  0.209563 -1.405213 -0.416534   
     H3K4me3          -1.670054 -0.107283  0.331703 -1.055807 -0.315625   
     H3K4ac1          -2.142428 -0.311632  0.062620 -2.282088 -0.404569   
...                         ...       ...       ...       ...       ...   
TGFb H4(20to23)K20me0 -0.220801  0.897532  0.053143  2.402090  1.897346   
     H4(20to23)K20me1 -0.237642 -0.289277  0.045437 -2.517549  0.315347   
     H4(20to23)K20me2  1.736994 -0.003563  0.980752 -0.025149 -0.513190   
     H4(20to23)K20me3  2.856347  0.380403  0.199495  1.441623 -0.151613   
     H4(68-78)AltNorm  0.703132  0.023534  0.757677  0.322969  0.107820   

                                           
                              p         t  
     histone                               
BMP2 H3K4me0           0.362895 -1.000734  
     H3K4me1           0.132213 -1.797320  
     H3K4me2           0.058478 -2.442434  
     H3K4me3           0.012928 -3.777087  
     H3K4ac1           0.029279 -3.024100  
...                         ...       ...  
TGFb H4(20to23)K20me0  0.008449  4.204937  
     H4(20to23)K20me1  0.027236  3.087512  
     H4(20to23)K20me2  0.033130 -2.916962  
     H4(20to23)K20me3  0.560541 -0.623090  
     H4(68-78)AltNorm  0.415230  0.887961  

[378 rows x 12 columns]

In [68]:
ligand_min_gcp = mergedPvalDf_gcp.groupby(level=['histone'],axis=0).min()

In [69]:
ligand_min_gcp

_4                            _8  \
                                     fc         p         t        fc   
histone                                                                 
H3.3K27me0K36me0              -0.022141  0.377868 -0.120410  0.058754   
H3K18ac0K23ac0                 0.008479  0.136263  0.157815 -0.031376   
H3K18ac0K23ac1                -0.160112  0.002582 -0.813113 -0.117078   
H3K18ac1K23ac0                -0.254294  0.071194 -2.188785 -0.327500   
H3K18ac1K23ac1                -0.363694  0.126502 -1.773547 -0.276511   
...                                 ...       ...       ...       ...   
H4(4to17)K5ac1me0             -0.233868  0.178788 -1.522185 -0.215321   
H4(4to17)K8ac1K12ac1K16ac1me0 -0.462149  0.071764 -2.183004 -0.712758   
H4(4to17)K8ac1K12ac1me0       -0.454331  0.037962 -2.651305 -0.395028   
H4(4to17)ac0me0               -0.214209  0.075709 -1.705528 -0.073881   
H4(68-78)AltNorm              -0.169134  0.113912 -1.849227  0.046619   

                                                        _24            \
                                      p         t        fc         p   
histone                                                                 
H3.3K27me0K36me0               0.111276  0.321436  0.204674  0.021550   
H3K18ac0K23ac0                 0.046308 -0.407225 -0.251543  0.000287   
H3K18ac0K23ac1                 0.120508 -1.808601 -0.940697  0.000142   
H3K18ac1K23ac0                 0.019032 -3.181806 -0.681219  0.001618   
H3K18ac1K23ac1                 0.203472 -1.427042 -0.663300  0.016510   
...                                 ...       ...       ...       ...   
H4(4to17)K5ac1me0              0.038038 -1.573512  0.276692  0.002410   
H4(4to17)K8ac1K12ac1K16ac1me0  0.005060 -4.305937 -1.427837  0.000070   
H4(4to17)K8ac1K12ac1me0        0.058962 -2.326007 -0.992695  0.000424   
H4(4to17)ac0me0                0.004328 -0.491726 -0.062355  0.028309   
H4(68-78)AltNorm               0.028799  0.245344 -0.019202  0.014429   

                                              _48                      
                                      t        fc         p         t  
histone                                                                
H3.3K27me0K36me0               1.169561  0.351294  0.006435  1.675614  
H3K18ac0K23ac0                -5.084937 -0.007113  0.000026 -0.040623  
H3K18ac0K23ac1                -8.532521 -0.550818  0.000156 -2.749932  
H3K18ac1K23ac0                -5.429911 -0.016292  0.027738 -0.122735  
H3K18ac1K23ac1                -3.295004 -0.444402  0.019866 -1.970630  
...                                 ...       ...       ...       ...  
H4(4to17)K5ac1me0              1.492897  0.113719  0.000618  0.789076  
H4(4to17)K8ac1K12ac1K16ac1me0 -9.668283 -0.516985  0.054608 -2.498230  
H4(4to17)K8ac1K12ac1me0       -6.999376 -0.433121  0.130125 -1.753136  
H4(4to17)ac0me0               -0.414067 -0.091280  0.001854 -0.578625  
H4(68-78)AltNorm              -0.248266 -0.170733  0.015068 -1.234054  

[63 rows x 12 columns]

In [70]:
hour_min_gcp = ligand_min_gcp.groupby(level=1, axis=1).min()

In [71]:
ligand_significants = hour_min_gcp[hour_min_gcp['p']<0.05].sort_values(by=['p'], ascending=False).drop(['fc', 't'], axis=1)
ligand_significants

,p
histone,
H3K27me2K36me1,4.915798e-02
H3K9me2K14ac0,4.158016e-02
H3K79me2,3.132840e-02
H4(20to23)K20me3,2.893986e-02
H3K18ac1K23ac1,1.651042e-02
H4(4to17)K12ac1me0,1.514015e-02
H4(68-78)AltNorm,1.442855e-02
H4(4to17)K5ac1K16ac1me0,1.395618e-02
H4(4to17)K16ac1me0,1.350588e-02


In [72]:
ligand_significants.to_excel("pval_005_gcp.xlsx")